In [130]:
# import apriltag
import cv2
import numpy as np
import copy
import matplotlib.pyplot as plt
import glob
import os
import time
from scipy.optimize import minimize, least_squares
from scipy.spatial.transform import Rotation as scipy_R

In [131]:
%matplotlib inline

In [132]:
left_serial = '22240376'
right_serial = '22206080'

# left_serial = '22206080'
# right_serial = '22240368'

# left_serial = '22240368'
# right_serial = '22240389'

# left_serial = '22240389'
# right_serial = '22222961'

# left_serial = '22222961'
# right_serial = '22240382' #

# left_serial = '22240382'
# right_serial = '22240376'


# left_top_corners = [[64, 228*2], [0, 228], [64, 0], [64, 100], [64, 228], [64, 288]] # w = 1920 h = 1080
serial_number_list = ['22240376', '22206080', '22240368', '22240389', '22222961', '22240382']

# lt_left = 

checker_side_length = 60

In [133]:
def calculate_Tr(K_np, rvec_np, tvec_np):
    _R_cv2bcam = np.array([[1,0,0],
                       [0,-1,0],
                       [0,0,-1]])
    R, _ = cv2.Rodrigues(rvec_np)
    # t = tvec_np[_i]/100. #(mm to M)? scale problem exist.
    t = tvec_np/100. #(mm to M)? scale problem exist.
    # k = first_K_np
    R_bcam= np.matmul(_R_cv2bcam, R)
    T_bcam = np.matmul(_R_cv2bcam, t)                       
    R_world = R_bcam.transpose()
    T_world = -np.matmul(R_world, T_bcam)

    R_world = R_world.reshape(3,3)
    T_world = T_world.reshape(3,1)

    Tr = np.identity(4)
    Tr[:3,:3] = R_world
    Tr[:3,3:] = T_world

    return Tr# debugging image is not nessesary, apriltag detection already confirmed.

In [134]:
checker_board_size = (9, 10) # (rows, cols)

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((checker_board_size[0]*checker_board_size[1],3), np.float32)
objp[:,:2] = np.mgrid[0:checker_board_size[0],0:checker_board_size[1]].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.


In [135]:
objp.shape

(90, 3)

# LOAD INTRINSIC RESULTS

In [136]:
# left_serial = '22222961'
# right_serial = '22240382'

# left_serial = '22240382'
# right_serial = '22240376'

# apriltag_one_side_length = 203


data_root = '/home/hict/yeonkunlee/MULTIVIEW_CAL_HICT/cal_image'
prefix = 'left_' + left_serial + '_right_' + right_serial

print(os.path.join(data_root, 'intrinsic_images_checker', left_serial + '_intrinsic_result.npz'))

left_npz_file = np.load(os.path.join(data_root, left_serial + '_intrinsic_result.npz'))
left_ret = left_npz_file['ret']
left_mtx = left_npz_file['mtx']
left_dist = left_npz_file['dist']
# left_rvecs = left_npz_file['rvecs']
# left_tvecs = left_npz_file['tvecs']

right_npz_file = np.load(os.path.join(data_root, right_serial + '_intrinsic_result.npz'))
right_ret = right_npz_file['ret']
right_mtx = right_npz_file['mtx']
right_dist = right_npz_file['dist']
# right_rvecs = right_npz_file['rvecs']
# right_tvecs = right_npz_file['tvecs']

print('ret = ', left_ret, right_ret)

/home/hict/yeonkunlee/MULTIVIEW_CAL_HICT/cal_image/intrinsic_images_checker/22240376_intrinsic_result.npz
ret =  0.8821481952934551 0.9254477066411001


# LOAD STEREO CALIBRATION IMAGE

In [137]:
# load stereo calibration image form directory

# /workspace/multiview_calibration/MULTIVIEW/cal_image/extrinsic_images/left_22240382_right_22240376
stereo_root = os.path.join(data_root, 'extrinsic_images_checker', prefix)

left_image_dir = os.path.join(stereo_root, 'left_'+left_serial)
left_image_list = glob.glob(os.path.join(left_image_dir, '*.png'))
left_image_list.sort()
right_image_dir = os.path.join(stereo_root, 'right_'+right_serial)
right_image_list = glob.glob(os.path.join(right_image_dir, '*.png'))
right_image_list.sort()

left_basename_list = [os.path.basename(x) for x in left_image_list]
right_basename_list = [os.path.basename(x) for x in right_image_list]
assert left_basename_list == right_basename_list, 'left and right image list are not matched'
assert len(left_basename_list) > 0, 'at least one image is needed'

In [138]:
len(left_basename_list)

39

In [139]:
# prepare parameters for cv2.stereoCalibrate
# prepare object points, image points of left, image points of right and image size
# read left, right image from directory

vis_factor = 0.5

num_image = len(left_image_list)
print('total num_image:', num_image)

april_objpoints = []
left_april_imgpoints = []
right_april_imgpoints = []
april_imagename = []

processing_counter = 0

for i in range(num_image):
    left_image = cv2.imread(left_image_list[i])
    right_image = cv2.imread(right_image_list[i])

    left_image_gray = cv2.cvtColor(left_image, cv2.COLOR_BGR2GRAY)
    right_image_gray = cv2.cvtColor(right_image, cv2.COLOR_BGR2GRAY)
    # find checker board
    left_success, left_corners = cv2.findChessboardCorners(left_image_gray, checker_board_size, None)
    right_success, right_corners = cv2.findChessboardCorners(right_image_gray, checker_board_size, None)


    if left_success and right_success:
        processing_counter += 1
    else:
        continue

    left_corners = np.array(left_corners)
    left_imgpoints = left_corners.reshape(-1, 1, 2)
    left_imgpoints = left_imgpoints.astype(np.float32)
    left_april_imgpoints.append(left_imgpoints)
    right_corners = np.array(right_corners)
    right_imgpoints = right_corners.reshape(-1, 1, 2)
    right_imgpoints = right_imgpoints.astype(np.float32)
    right_april_imgpoints.append(right_imgpoints)

    apriltag_objp = np.array(objp)
    apriltag_objp *= checker_side_length
    april_objpoints.append(apriltag_objp.astype(np.float32))
    april_imagename.append([left_image_list[i], right_image_list[i]])

print('processed image number:', processing_counter)

total num_image: 39
processed image number: 33


In [140]:
# save objpoints, imgpoints to file
save_dir = os.path.join(stereo_root, 'calibration_data.npz')
print(save_dir)
np.savez(save_dir, april_objpoints=april_objpoints, left_april_imgpoints=left_april_imgpoints, right_april_imgpoints=right_april_imgpoints, april_imagename=april_imagename)

/home/hict/yeonkunlee/MULTIVIEW_CAL_HICT/cal_image/extrinsic_images_checker/left_22240376_right_22206080/calibration_data.npz


In [141]:
# # randomly sample a subset of april_imgpoints and april_objpoints and april_imagename. no duplicated.
# # num_sample = processing_counter # just use all images for now.
# num_sample = processing_counter
# total_num = len(april_objpoints)
# assert total_num >= num_sample, 'total number of april_imgpoints is less than num_sample'
# sample_idx = np.random.choice(total_num, num_sample, replace=False)
# left_april_imgpoints_sampled = [left_april_imgpoints[i] for i in sample_idx]
# right_april_imgpoints_sampled = [right_april_imgpoints[i] for i in sample_idx]
# april_objpoints_sampled = [april_objpoints[i] for i in sample_idx]
# april_imagename_sampled = [april_imagename[i] for i in sample_idx]


In [142]:
# # just intrinsic calibration for debug
# left_ret, left_mtx, left_dist, left_rvecs, left_tvecs = cv2.calibrateCamera(april_objpoints, left_april_imgpoints, left_image.shape[:2][::-1], None, None)
# right_ret, right_mtx, right_dist, right_rvecs, right_tvecs = cv2.calibrateCamera(april_objpoints, right_april_imgpoints, right_image.shape[:2][::-1], None, None)
# # 1.5343584670449189
# # [[2.20554484e+03 0.00000000e+00 9.85254599e+02]
# #  [0.00000000e+00 2.21890933e+03 7.94500597e+02]
# #  [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

In [143]:
print(left_ret)
print(right_ret)

0.8821481952934551
0.9254477066411001


In [144]:
# np.savez(os.path.join(data_root, left_serial + '_intrinsic_result.npz'), ret=left_ret, mtx=left_mtx, dist=left_dist, rvecs=left_rvecs, tvecs=left_tvecs)
# # np.savez(os.path.join(data_root, right_serial + '_intrinsic_result.npz'), ret=right_ret, mtx=right_mtx, dist=right_dist, rvecs=right_rvecs, tvecs=right_tvecs)

In [145]:
#change this if stereo calibration not good.
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
stereocalibration_flags = cv2.CALIB_FIX_INTRINSIC
(width, height) = left_image.shape[:2][::-1]
print(width, height)

1920 1080


In [146]:
ret, CM1, dist1, CM2, dist2, R, T, E, F = cv2.stereoCalibrate(april_objpoints, left_april_imgpoints, right_april_imgpoints, left_mtx, left_dist,
                                                                 right_mtx, right_dist, (width, height), criteria = criteria, flags = stereocalibration_flags)

In [147]:
print('ret:', ret)
print('CM1:', CM1)
print('dist1:', dist1)
print('CM2:', CM2)
print('dist2:', dist2)
print('R:', R)
print('T:', T)
print('E:', E)
print('F:', F)

ret: 0.7622786904053827
CM1: [[1.06067353e+03 0.00000000e+00 9.53566123e+02]
 [0.00000000e+00 1.05945771e+03 5.48194101e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist1: [[-0.04766772  0.11490369  0.001382    0.00096083 -0.05987259]]
CM2: [[1.05280920e+03 0.00000000e+00 9.37978538e+02]
 [0.00000000e+00 1.05053001e+03 5.79589049e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist2: [[-0.05928633  0.1401007   0.00189698  0.00073395 -0.0787084 ]]
R: [[ 0.12078356 -0.63190183  0.76557914]
 [ 0.68047661  0.61421672  0.39961157]
 [-0.72274679  0.47269219  0.50418169]]
T: [[-3609.4258905 ]
 [-1817.67844049]
 [ 2095.13830556]]
E: [[ -111.97135313 -2146.07137469 -1753.68169953]
 [-2355.64270864   382.2256961   3423.80062575]
 [-2236.58421668 -3365.56405692   -50.79165786]]
F: [[-8.10894131e-08 -1.55596384e-06 -4.16774348e-04]
 [-1.70965256e-06  2.77725928e-07  4.11367451e-03]
 [-6.38310075e-04 -1.27049893e-03  1.00000000e+00]]


In [148]:
stereo_root

'/home/hict/yeonkunlee/MULTIVIEW_CAL_HICT/cal_image/extrinsic_images_checker/left_22240376_right_22206080'

In [149]:
rvec_cam = scipy_R.from_matrix(R).as_rotvec().reshape(3,1)
tvec_cam = T.reshape(3,1)
np.save(os.path.join(stereo_root, 'left_' + left_serial + '_right_' + right_serial+'_rvec.npy'), rvec_cam)
np.save(os.path.join(stereo_root, 'left_' + left_serial + '_right_' + right_serial+'_tvec.npy'), tvec_cam)

In [150]:
def distance_calculator(_rvec_cam_left, _tvec_cam_left, _rvec_cam_right, _tvec_cam_right, _april_objpoints, _left_april_imgpoints, _right_april_imgpoints, _left_mtx, _left_dist, _right_mtx, _right_dist):
    distance_holder = []
    for i in range(len(_april_objpoints)):
        left_success, left_rvec, left_tvec = cv2.solvePnP(_april_objpoints[i], _left_april_imgpoints[i], _left_mtx, _left_dist, 0)
        left_R = cv2.Rodrigues(left_rvec)[0]
        right_success, right_rvec, right_tvec = cv2.solvePnP(_april_objpoints[i], _right_april_imgpoints[i], _right_mtx, _right_dist, 0)
        right_R = cv2.Rodrigues(right_rvec)[0]

        # left_detected_imgpoints = _left_april_imgpoints[i].reshape(-1, 2)
        # right_detected_imgpoints = _right_april_imgpoints[i].reshape(-1, 2)

        left_object_points = (left_R.dot(_april_objpoints[i].T) + left_tvec).T # N, 3
        right_object_points = (right_R.dot(_april_objpoints[i].T) + right_tvec).T # N, 3

        R_cam_left = cv2.Rodrigues(_rvec_cam_left)[0]
        left_object_points = R_cam_left.T.dot(left_object_points.T - _tvec_cam_left).T

        R_cam_right = cv2.Rodrigues(_rvec_cam_right)[0]
        right_object_points = R_cam_right.T.dot(right_object_points.T - _tvec_cam_right).T

        assert left_object_points.shape == right_object_points.shape

        distance = np.mean(np.linalg.norm(left_object_points - right_object_points, axis=1))

        distance_holder.append(distance)
    return np.mean(distance_holder)

In [151]:
# sample_i = 5
# one_left_imagename, one_right_imagename = april_imagename[sample_i]
# print(one_left_imagename)
# one_april_objpoints = april_objpoints[sample_i]
# print(one_april_objpoints.shape)

# one_left_april_imgpoints = left_april_imgpoints[sample_i]
# one_right_april_imgpoints = right_april_imgpoints[sample_i]

# left_image = cv2.imread(one_left_imagename)
# right_image = cv2.imread(one_right_imagename)

# # project 3D points to 2D points
# # left_imgpoints = cv2.projectPoints(one_april_objpoints, np.zeros([3,1]), np.zeros([3,1]), left_mtx, left_dist)
# # right_imgpoints = cv2.projectPoints(one_april_objpoints, rvec_cam, tvec_cam, right_mtx, right_dist)


In [152]:
distance_calculator(np.zeros([3,1]), np.zeros([3,1]), rvec_cam, tvec_cam, april_objpoints, left_april_imgpoints, right_april_imgpoints, left_mtx, left_dist, right_mtx, right_dist)

11.151263668386212

In [153]:
# 19.8
# 15.7
# 21.2
# 27.6
# 25.8
# 12.04